In [ ]:
import os
from datetime import datetime
import random
import math
import numpy as np
import pandas as pd
import pprint
import gzip
import csv
import logging
from IPython.display import display
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import zipfile

SNLI_DATASET_PATH = '/content/drive/MyDrive/prediction-with-assortment/SNLI/snli_1.0.zip'

SNLI_DATASET_EXTRACT_DIR = '/content/drive/MyDrive/prediction-with-assortment/SNLI'

SNLI_DATASET_DIR = os.path.join(SNLI_DATASET_EXTRACT_DIR, "snli_1.0")

# Open the zip file in read mode
with zipfile.ZipFile(SNLI_DATASET_PATH, 'r') as zip_ref:
    # Extract all files to the specified directory
    zip_ref.extractall(SNLI_DATASET_EXTRACT_DIR)

os.listdir(SNLI_DATASET_DIR)

['.DS_Store',
 'Icon\r',
 'README.txt',
 'snli_1.0_dev.jsonl',
 'snli_1.0_dev.txt',
 'snli_1.0_test.jsonl',
 'snli_1.0_test.txt',
 'snli_1.0_train.jsonl',
 'snli_1.0_train.txt']

In [5]:
SNLI_TRAIN_FILE =  os.path.join(SNLI_DATASET_DIR, "snli_1.0_train.txt")
SNLI_DEV_FILE = os.path.join(SNLI_DATASET_DIR, "snli_1.0_dev.txt")
SNLI_TEST_FILE = os.path.join(SNLI_DATASET_DIR, "snli_1.0_dev.txt")

df_train = pd.read_csv(SNLI_TRAIN_FILE, sep="\t")
df_dev = pd.read_csv(SNLI_DEV_FILE, sep="\t")
df_test = pd.read_csv(SNLI_TEST_FILE, sep="\t")
# The SNLI dataset contains several columns, but for many tasks, only gold_label, sentence1, and sentence2 are needed.

df_train[['sentence1', 'sentence2', 'gold_label']][:5]

,sentence1,sentence2,gold_label
0,A person on a horse jumps over a broken down a...,A person is training his horse for a competition.,neutral
1,A person on a horse jumps over a broken down a...,"A person is at a diner, ordering an omelette.",contradiction
2,A person on a horse jumps over a broken down a...,"A person is outdoors, on a horse.",entailment
3,Children smiling and waving at camera,They are smiling at their parents,neutral
4,Children smiling and waving at camera,There are children present,entailment
